## Training Task Adapters
Using randomized search, we identify optimal hyperparameters to train task specfic adapters on GLUE tasks.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive/cs7643-deep-learning-summer-2021/

/content/drive/MyDrive/cs7643-deep-learning-summer-2021


In [3]:
!pip install -Uqq adapter-transformers datasets

%load_ext autoreload
%autoreload 2
from task_utils import TaskModelArguments, TaskDataTrainingArguments, task_to_keys
from task import train_task_adapter
from transformers import (
    MultiLingAdapterArguments,
    TrainingArguments,
)

     |████████████████████████████████| 2.5 MB 7.8 MB/s 
     |████████████████████████████████| 542 kB 52.2 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 895 kB 53.5 MB/s 
     |████████████████████████████████| 3.3 MB 51.3 MB/s 
     |████████████████████████████████| 243 kB 68.2 MB/s 
     |████████████████████████████████| 76 kB 5.9 MB/s 
     |████████████████████████████████| 118 kB 73.0 MB/s 


### CoLA Task

In [4]:
def initParse(dictionary):
  model = TaskModelArguments(
      model_name_or_path=dictionary.get('model_name_or_path')
  )

  data = TaskDataTrainingArguments(
      task_name=dictionary.get('task_name'),
      max_seq_length=dictionary.get('max_seq_length'),
      pad_to_max_length=dictionary.get('pad_to_max_length')
  )

  training = TrainingArguments(
    adam_beta1=dictionary.get('adam_beta1'),
    adam_beta2=dictionary.get('adam_beta2'),
    adam_epsilon=dictionary.get('adam_epsilon'),
    learning_rate=dictionary.get('learning_rate'),
    warmup_ratio=dictionary.get('warmup_ratio'),
    warmup_steps=dictionary.get('warmup_steps'),
    weight_decay=dictionary.get('weight_decay'),
    do_train=dictionary.get('do_train'),
    do_eval=dictionary.get('do_train'),
    num_train_epochs=dictionary.get('num_train_epochs'), # CHANGE ME
    overwrite_output_dir=dictionary.get('overwrite_output_dir'),
    output_dir=f"./adapter/task/{dictionary.get('task_name')}",
  )

  adapter = MultiLingAdapterArguments(
      train_adapter=True,
      adapter_config="pfeiffer",
  )

  return model, data, training, adapter

In [30]:
import random
import itertools
import subprocess
import pandas as pd

def getParams(dictionary, limit):
  paramsList = [dict(zip(dictionary, v)) for v in itertools.product(*dictionary.values())]
  random.shuffle(paramsList)

  if limit is not False:
    paramsList = paramsList[0:min(limit, len(paramsList))]

  return paramsList

**Define Dictionary of Hyperparameters**

In [6]:
task = 'cola'
paramDictionary = {'task_name':[task],
                   'model_name_or_path':['roberta-base'],
                   'max_seq_length':[64,128,256],
                   'pad_to_max_length':[True],
                   #'per_device_train_batch_size':[32,64,128],
                   'adam_beta1':[.9,.99,.999],
                   'adam_beta2':[.999, .99, .9],
                   'adam_epsilon':[1e-8,1e-7,1e-6],
                   'learning_rate':[1e-4,1e-4],
                   'warmup_ratio':[0.0],
                   'warmup_steps':[0],
                   'weight_decay':[0.0],
                   'do_train':[True],
                   'do_eval':[True],
                   'num_train_epochs':[1],
                   'overwrite_output_dir':[True],
                   'adapter_config':['pfeiffer']
                   }

**Begin Looping**

In [33]:
limit = 5 #Numerical or False for no limit
paramsList = getParams(paramDictionary,limit)
output=[]
for p in paramsList:
  model, data, training, adapter = initParse(p)
  row = []
  row.extend(list(p.values()))
  train_stats, eval_stats = train_task_adapter(
    model_args=model, adapter_args=adapter, training_args=training, data_args=data
    )
  row.extend(list(train_stats.values()))
  row.extend(list(eval_stats.values()))

  output.append(row)

header = []
header.extend(list(p.keys()))
header.extend(list(train_stats.keys()))
header.extend(list(eval_stats.keys()))

output = pd.DataFrame(output, columns = header)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
07/26/2021 21:28:29 - WARNING - task -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
07/26/2021 21:28:29 - INFO - task -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.99,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=500,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gr

Step,Training Loss
500,0.561700
1000,0.477400


Saving model checkpoint to ./adapter/task/cola/checkpoint-500
Configuration saved in ./adapter/task/cola/checkpoint-500/cola/adapter_config.json
Module weights saved in ./adapter/task/cola/checkpoint-500/cola/pytorch_adapter.bin
Configuration saved in ./adapter/task/cola/checkpoint-500/cola/head_config.json
Module weights saved in ./adapter/task/cola/checkpoint-500/cola/pytorch_model_head.bin
Configuration saved in ./adapter/task/cola/checkpoint-500/cola/head_config.json
Module weights saved in ./adapter/task/cola/checkpoint-500/cola/pytorch_model_head.bin
tokenizer config file saved in ./adapter/task/cola/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./adapter/task/cola/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./adapter/task/cola/checkpoint-1000
Configuration saved in ./adapter/task/cola/checkpoint-1000/cola/adapter_config.json
Module weights saved in ./adapter/task/cola/checkpoint-1000/cola/pytorch_adapter.bin
Configuration saved in ./adap

07/26/2021 21:31:01 - INFO - task -   ***** Eval results cola *****
07/26/2021 21:31:01 - INFO - task -     eval_loss = 0.5537135004997253
07/26/2021 21:31:01 - INFO - task -     eval_matthews_correlation = 0.40058872203446944
07/26/2021 21:31:01 - INFO - task -     eval_runtime = 8.4217
07/26/2021 21:31:01 - INFO - task -     eval_samples_per_second = 123.847
07/26/2021 21:31:01 - INFO - task -     eval_steps_per_second = 15.555
07/26/2021 21:31:01 - INFO - task -     epoch = 1.0
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
07/26/2021 21:31:01 - WARNING - task -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
07/26/2021 21:31:01 - INFO - task -   Training/evaluation parameters TrainingArgum

Step,Training Loss
500,0.582700
1000,0.502600


...

  0%|          | 0/2 [00:00<?, ?ba/s]

07/26/2021 21:37:50 - WARNING - datasets.arrow_dataset -   Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-ef1f0fc55fd588c6.arrow
07/26/2021 21:37:50 - INFO - task -   Sample 1824 of the training set: {'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Step,Training Loss
500,0.587700
1000,0.506400


Saving model checkpoint to ./adapter/task/cola/checkpoint-500
Configuration saved in ./adapter/task/cola/checkpoint-500/cola/adapter_config.json
Module weights saved in ./adapter/task/cola/checkpoint-500/cola/pytorch_adapter.bin
Configuration saved in ./adapter/task/cola/checkpoint-500/cola/head_config.json
Module weights saved in ./adapter/task/cola/checkpoint-500/cola/pytorch_model_head.bin
Configuration saved in ./adapter/task/cola/checkpoint-500/cola/head_config.json
Module weights saved in ./adapter/task/cola/checkpoint-500/cola/pytorch_model_head.bin
tokenizer config file saved in ./adapter/task/cola/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./adapter/task/cola/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./adapter/task/cola/checkpoint-1000
Configuration saved in ./adapter/task/cola/checkpoint-1000/cola/adapter_config.json
Module weights saved in ./adapter/task/cola/checkpoint-1000/cola/pytorch_adapter.bin
Configuration saved in ./adap

07/26/2021 21:43:14 - INFO - task -   ***** Eval results cola *****
07/26/2021 21:43:14 - INFO - task -     eval_loss = 0.6077342629432678
07/26/2021 21:43:14 - INFO - task -     eval_matthews_correlation = 0.36704088018637543
07/26/2021 21:43:14 - INFO - task -     eval_runtime = 18.4852
07/26/2021 21:43:14 - INFO - task -     eval_samples_per_second = 56.424
07/26/2021 21:43:14 - INFO - task -     eval_steps_per_second = 7.087
07/26/2021 21:43:14 - INFO - task -     epoch = 1.0


In [34]:
output

,task_name,model_name_or_path,max_seq_length,pad_to_max_length,adam_beta1,adam_beta2,adam_epsilon,learning_rate,warmup_ratio,warmup_steps,weight_decay,do_train,do_eval,num_train_epochs,overwrite_output_dir,adapter_config,train_runtime,train_samples_per_second,train_steps_per_second,total_flos,train_loss,epoch,eval_loss,eval_matthews_correlation,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
0,cola,roberta-base,128,True,0.900,0.990,1.000000e-08,0.0001,0.0,0,0.0,True,True,1,True,pfeiffer,136.7348,62.537,7.818,8.283319e+14,0.513692,1.0,0.553714,0.400589,8.4217,123.847,15.555,1.0
1,cola,roberta-base,128,True,0.990,0.900,1.000000e-06,0.0001,0.0,0,0.0,True,True,1,True,pfeiffer,143.5196,59.581,7.448,8.283319e+14,0.535596,1.0,0.537808,0.388289,8.7025,119.851,15.053,1.0
2,cola,roberta-base,128,True,0.999,0.900,1.000000e-07,0.0001,0.0,0,0.0,True,True,1,True,pfeiffer,145.8208,58.640,7.331,8.283319e+14,0.549654,1.0,0.554844,0.347854,8.7829,118.754,14.915,1.0
3,cola,roberta-base,64,True,0.999,0.990,1.000000e-08,0.0001,0.0,0,0.0,True,True,1,True,pfeiffer,74.8210,114.286,14.287,4.141660e+14,0.548469,1.0,0.556882,0.357428,4.4919,232.196,29.164,1.0
4,cola,roberta-base,256,True,0.999,0.999,1.000000e-08,0.0001,0.0,0,0.0,True,True,1,True,pfeiffer,304.1001,28.119,3.515,1.656664e+15,0.542120,1.0,0.607734,0.367041,18.4852,56.424,7.087,1.0
